In [1]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
df = pd.read_csv('dns.csv', encoding = "ISO-8859-1")

In [3]:
df.head()

,QR,OPCODE,QNAME,QTYPE,QCLS,RRNAME,RRTYPE,RCLS,TTL,RLEN,RDATA,AA,ID,QDCNT,ANCNT,NSCNT,ARCNT
0,0,0,planetlab2.isi.jhu.edu,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0,41945,1,0,0,1
1,1,0,planetlab2.isi.jhu.edu,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,41945,1,0,1,1
2,0,0,planetlab2.isi.jhu.edu,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0,25281,1,0,0,1
3,1,0,planetlab2.isi.jhu.edu,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,25281,1,0,1,1
4,0,0,dns2.isi.jhu.edu,255.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0,9882,1,0,0,0


In [4]:
# check where QNAME is NaN and get rid of those rows
display('{} rows with no QNAME out of {} total rows'.format(len(df[df['QNAME'].notnull() == False].index), len(df.index)))
df = df[df['QNAME'].notnull() == True]

'53 rows with no QNAME out of 404198 total rows'

In [5]:
def determine_empty_row_count(df, column_name):
    display('{} rows with no {} out of {} total rows'.format(len(df[df[column_name].notnull() == False].index),
                                                             column_name, len(df.index)))
for column in df.columns:
    determine_empty_row_count(df, column)

'0 rows with no QR out of 404145 total rows'

'0 rows with no OPCODE out of 404145 total rows'

'0 rows with no QNAME out of 404145 total rows'

'0 rows with no QTYPE out of 404145 total rows'

'0 rows with no QCLS out of 404145 total rows'

...

'0 rows with no ID out of 404145 total rows'

'0 rows with no QDCNT out of 404145 total rows'

'0 rows with no ANCNT out of 404145 total rows'

'0 rows with no NSCNT out of 404145 total rows'

'0 rows with no ARCNT out of 404145 total rows'

In [6]:
df = df[df['QTYPE'].notnull() == True]
df = df[df['QCLS'].notnull() == True]

In [7]:
df.tail()

,QR,OPCODE,QNAME,QTYPE,QCLS,RRNAME,RRTYPE,RCLS,TTL,RLEN,RDATA,AA,ID,QDCNT,ANCNT,NSCNT,ARCNT
404193,1,0,www.weather.com,1.0,1.0,www.weather.com,5.0,1.0,60.0,33.0,pmd-wwwweathercomedgekeynet,0,27818,1,3,0,0
404194,0,0,textsecure-service-ca.whispersystems.org,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0,540,1,0,0,0
404195,0,0,textsecure-service-ca.whispersystems.org,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0,32517,1,0,0,1
404196,1,0,textsecure-service-ca.whispersystems.org,1.0,1.0,textsecure-service-ca.whispersystems.org,1.0,1.0,60.0,4.0,6ìvÿ,0,32517,1,1,0,1
404197,1,0,textsecure-service-ca.whispersystems.org,1.0,1.0,textsecure-service-ca.whispersystems.org,1.0,1.0,60.0,4.0,6ìvÿ,0,540,1,1,0,0


In [8]:
# scikit-learn can only handle numerical data
# we can drop the rdata column for running the algorithm for now, but we can also turn it into ints somehow
# rlen is more telling: we don't actually care what the data is for now
# -----------
# we need to map qname to unique integers for the algorithm. the same needs to happen for rrname
qname_unique = df['QNAME'].dropna().unique()
rrname_unique = df['RRNAME'].dropna().unique()
display('There are {} unique QNAMEs and {} unique RRNAMEs'.format(len(qname_unique), len(rrname_unique)))
all_names = set(qname_unique) | set(rrname_unique)
display('There are {} total unique names'.format(len(all_names)))
mapping = {name: i for name, i in zip(all_names, range(len(all_names)))}

'There are 11226 unique QNAMEs and 9780 unique RRNAMEs'

'There are 11226 total unique names'

In [9]:
#df.replace(mapping, inplace=True)
for name in mapping:
    df.loc[df['QNAME'] == name] = mapping[name]
    df.loc[df['RRNAME'] == name] = mapping[name]
display(df.head())

,QR,OPCODE,QNAME,QTYPE,QCLS,RRNAME,RRTYPE,RCLS,TTL,RLEN,RDATA,AA,ID,QDCNT,ANCNT,NSCNT,ARCNT
0,10510,10510,10510,10510.0,10510.0,10510,10510.0,10510.0,10510.0,10510.0,10510,10510,10510,10510,10510,10510,10510
1,10510,10510,10510,10510.0,10510.0,10510,10510.0,10510.0,10510.0,10510.0,10510,10510,10510,10510,10510,10510,10510
2,10510,10510,10510,10510.0,10510.0,10510,10510.0,10510.0,10510.0,10510.0,10510,10510,10510,10510,10510,10510,10510
3,10510,10510,10510,10510.0,10510.0,10510,10510.0,10510.0,10510.0,10510.0,10510,10510,10510,10510,10510,10510,10510
4,2629,2629,2629,2629.0,2629.0,2629,2629.0,2629.0,2629.0,2629.0,2629,2629,2629,2629,2629,2629,2629


In [10]:
df.head()

,QR,OPCODE,QNAME,QTYPE,QCLS,RRNAME,RRTYPE,RCLS,TTL,RLEN,RDATA,AA,ID,QDCNT,ANCNT,NSCNT,ARCNT
0,10510,10510,10510,10510.0,10510.0,10510,10510.0,10510.0,10510.0,10510.0,10510,10510,10510,10510,10510,10510,10510
1,10510,10510,10510,10510.0,10510.0,10510,10510.0,10510.0,10510.0,10510.0,10510,10510,10510,10510,10510,10510,10510
2,10510,10510,10510,10510.0,10510.0,10510,10510.0,10510.0,10510.0,10510.0,10510,10510,10510,10510,10510,10510,10510
3,10510,10510,10510,10510.0,10510.0,10510,10510.0,10510.0,10510.0,10510.0,10510,10510,10510,10510,10510,10510,10510
4,2629,2629,2629,2629.0,2629.0,2629,2629.0,2629.0,2629.0,2629.0,2629,2629,2629,2629,2629,2629,2629
